In [1]:
import json
import gc
import random
import networkx as nx
from networkx.readwrite import json_graph
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import scml
import mylib

In [2]:
# window length to the left or right context
m = 2
iters = 1

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(31)

In [4]:
%%time
with open("input/vocab2.json") as f:
    id2label = [w for w, c in json.load(f)]
label2id = {k:v for v, k in enumerate(id2label)}
print(f"len(label2id)={len(label2id):,}")

len(label2id)=3,748,278
Wall time: 4.39 s


In [5]:
del id2label
gc.collect()

21

In [6]:
%%time
with open("input/graph.json") as f:
    g = nx.adjacency_graph(json.load(f))
print(g)

DiGraph with 3747723 nodes and 99004731 edges
Wall time: 8min 50s


In [9]:
rows = []
for i in range(iters):
    for center in tqdm(g.nodes, desc=f"i={i}"):
        curr = center
        for _ in range(m):
            choices = []
            for u, v, d in g.in_edges(curr, data=True):
                #print(f"u={u}, v={v}, d={d}")
                choices.append((u, d["weight"]))
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            rows.append({
                "center": label2id[center],
                "outside": label2id[curr]
            })
        curr = center
        for _ in range(m):
            choices = []
            for u, v, d in g.out_edges(curr, data=True):
                choices.append((v, d["weight"]))
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            rows.append({
                "center": label2id[center],
                "outside": label2id[curr]
            })

i=0: 100%|██████████████████| 3747723/3747723 [2:36:04<00:00, 400.19it/s]


In [14]:
df = pd.DataFrame.from_records(rows)
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
cols = ["center", "outside"]
df[cols] = df[cols].astype(np.int32)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14461968 entries, 0 to 14461967
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   center   int32
 1   outside  int32
dtypes: int32(2)
memory usage: 110.3 MB


In [15]:
df.describe(percentiles=percentiles)

,center,outside
count,1.446197e+07,1.446197e+07
mean,1.846924e+06,7.173861e+05
std,1.075523e+06,8.564725e+05
min,0.000000e+00,0.000000e+00
1%,3.646300e+04,2.050000e+02
5%,1.825360e+05,3.050000e+03
10%,3.654940e+05,1.112300e+04
20%,7.320464e+05,4.343400e+04
30%,1.099470e+06,1.033570e+05
40%,1.467838e+06,2.014650e+05


In [16]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/pairs.parquet", index=False)

Wall time: 421 ms


In [17]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

RuntimeError: Not started